## train test 나누기

In [2]:
# 이미지 , json 같이 있는 폴더 train,val,test 나누기 (마지막 폴더 안에 클래스 구분 폴더 있어야 함)
import splitfolders
splitfolders.ratio('클래스 구분 윗 폴더 경로', output='결과 경로', seed=2022, ratio=(0.8, 0.1, 0.1), group_prefix=2)

Copying files: 534 files [00:01, 365.29 files/s]


## json to yolo txt

In [4]:
import os
from os import walk, getcwd
from PIL import Image
import json

In [5]:
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)
    
    

In [ ]:
"""-------------------------------------------------------------------""" 

""" Configure Paths"""   
mypath = "" # train img 경로
outpath = "" # train labels 경로
json_backup ="" # json 백업할 경로

wd = getcwd()
list_file = open('%s_list.txt'%(wd), 'w')

""" Get input json file list """
json_name_list = []
for file in os.listdir(mypath):
    if file.endswith(".json"):
        json_name_list.append(file)

""" Process """
for json_name in json_name_list:
    txt_name = json_name.rstrip(".json") + ".txt"
    """ Open input text files """
    txt_path = mypath + json_name
    print("Input:" + txt_path)
    txt_file = open(txt_path, "r")

    """ Open output text files """
    txt_outpath = outpath + txt_name
    print("Output:" + txt_outpath)
    txt_outfile = open(txt_outpath, "a")

    """ Convert the data to YOLO format """ 
    #lines = txt_file.read().split('\n')   #for ubuntu, use "\r\n" for windows use "\n"
    js_dict = json.load(txt_file)
    ############################################################################################
    for idx, box_list in enumerate(js_dict['shapes']):
        x1 = box_list['points'][0][0]
        y1 = box_list['points'][0][1]
        x2 = box_list['points'][1][0]
        y2 = box_list['points'][1][1]

        # 라벨을 숫자로 지정
        if box_list['label'] == 'ps':
            cls = '0'
	    #in case when labelling, points are not in the right order
        xmin = min(x1,x2)
        xmax = max(x1,x2)
        ymin = min(y1,y2)
        ymax = max(y1,y2)
        img_path = str('%s%s.jpg'%(mypath, os.path.splitext(json_name)[0]))

        im=Image.open(img_path)
        w= int(im.size[0])
        h= int(im.size[1])

        print(w, h)
        print(xmin, xmax, ymin, ymax)
        b = (xmin, xmax, ymin, ymax)
        bb = convert((w,h), b)
        print(bb)
        txt_outfile.write(cls + " " + " ".join([str(a) for a in bb]) + '\n')

    os.rename(txt_path,json_backup+json_name)	#move json file to backup folder

    """ Save those images with bb into list"""
    if(txt_file.read().count("label") != 0):
        list_file.write('%s/dataset/%s.jpg\n'%(wd, os.path.splitext(txt_name)[0]))
            
list_file.close()      